# Project Description

## Image Classification Using Known CNN Models

### Overview

In this project, we classify images using five well-known Convolutional Neural Network (CNN) models implemented with the Python `keras` library. The models used are `ResNet50`, `VGG16`, `InceptionV3`, `Xception`, and `EfficientNetB7`. The goal is to load an image, pass it through each of these models, and obtain the top prediction for the image. This project consists of two Python scripts: one for defining the CNN models (`cnn_models.py`) and one main script (`main.py`) for classifying an image.

### Project Components

#### 1. `cnn_models.py`

This script defines a class, `cnnModels`, which provides an interface to load and use the pre-trained CNN models. The class includes methods for initializing models, retrieving models by name, and classifying images.

##### `cnnModels` Class

- **`__init__(self)`**: Initializes the class and loads the pre-trained models.
- **`resnet(self)`**: Loads and returns the `ResNet50` model with ImageNet weights.
- **`vggnet(self)`**: Loads and returns the `VGG16` model with ImageNet weights.
- **`inception(self)`**: Loads and returns the `InceptionV3` model with ImageNet weights.
- **`convnet(self)`**: Loads and returns the `Xception` model with ImageNet weights.
- **`efficientnet(self)`**: Loads and returns the `EfficientNetB7` model with ImageNet weights.
- **`get_model(self, name)`**: Retrieves a model by name from the dictionary of models.
- **`classify_image(self, name, img)`**: Classifies an image using the specified model and returns the top 3 predictions.

#### 2. `main.ipynb`

This script demonstrates how to use the `cnnModels` class to classify an image.

##### Example Usage

```python
from cnn_models import cnnModels
from keras.preprocessing.image import load_img

# Specify the image path
img_path = './imgs/dog.jpeg'
img = load_img(img_path)

# Initialize the cnnModels class
model = cnnModels()

# Classify the image using ResNet50
preds1 = model.classify_image('ResNet50', img)

# Print the top predictions
for pred in preds1:
    print(f"{pred[1]}: {pred[2]}, {pred[3]}")


The state-of-the-art CNN models are tested using two datasets: 
1) AI-generated Images that contains 10 images
2) 10 Real Images collected from the internet

average accuracy, precision and recall scores. 

In [8]:
import cnn_models
import pandas as pd
from keras.utils import load_img #type: ignore
import os
import time
import psutil

In [ ]:
""" def get_predictions(image_dir):
    model = cnn_models.cnnModels()
    model_name = ['ResNet50', 'VGGNet16', 'InceptionV3', 'ConvNeXt', 'EfficientNet']
    result_df = pd.DataFrame(columns = model_name + [name + '_prob' for name in model_name])

    labels =[]    
    row_values = []

    for filename in os.listdir(image_dir):
        if filename.endswith('.jpeg') or filename.endswith('.png')or filename.endswith('.jpg'):
            image_path = os.path.join(image_dir, filename)
            img = load_img(image_path)   
            labels.append(filename.split('.')[0])
            prob_preds = []
            class_preds = []
            for name in model_name:
                preds = model.classify_image(name, img)[0][0][1:3] # returns top prediction in this format: [[('n02123045', 'tabby', 0.89)]], so we will get 1 and 2 value only.
                class_preds.append(preds[0]) # eg. tabby
                prob_preds.append(preds[1]) # eg. 0.89
            
            row_values.append(class_preds + prob_preds) # Append all predictions from 5 models into a single row.
    
    result_df = pd.DataFrame(row_values, columns = model_name + [name + '_prob' for name in model_name]) 
    result_df['label'] = labels        
    
    return result_df """

model_name = ['ResNet50', 'VGGNet16', 'InceptionV3', 'ConvNeXt', 'EfficientNet']
def get_predictions(image_dir):
    model = cnn_models.cnnModels()
    #model_name = ['ResNet50', 'VGGNet16', 'InceptionV3', 'ConvNeXt', 'EfficientNet']

    labels = []
    row_values = []

    for filename in os.listdir(image_dir):
        if filename.lower().endswith(('.jpeg', '.png', '.jpg')):
            image_path = os.path.join(image_dir, filename)
            img = load_img(image_path)
            base_name = filename.split('.')[0]
            labels.append(base_name)

            # extract the label from filename
            true_label = base_name.split('_')[0].lower()  # bike.jpg

            row_value = []

            for name in model_name:
                
                start_time = time.time()
                # Get the top 3 predictions
                top_preds = model.classify_image(name, img, top=3)[0]
                
                # Calcuate inference time / model size / memory usage
                inference_time = time.time() - start_time
                model_size = model.get_model(name).count_params()
                mem_usage = (model_size * 4) / (1024 ** 2)  # Memory in MB (float32=4 bytes)

                # Store prediction labels and probs
                for pred in top_preds:
                    row_value.append(pred[1])        # class label
                    row_value.append(round(pred[2], 4))  # prediction probability
                    row_value.append(1 if pred[1].lower() == true_label else 0)

                # Add inference time, model size, memory
                row_value.append(round(inference_time, 4))
                row_value.append(model_size)
                row_value.append(mem_usage)
                
            row_values.append(row_value)

    # Step 3: Define column headers (for top-3 predictions + accuracy + inference time + size + memory )
    result_columns = []
    for name in model_name:
        for i in range(1, 4):
            result_columns.append(f'{name}_top{i}')
            result_columns.append(f'{name}_prob{i}')
            result_columns.append(f'{name}_top{i}_acc')
        result_columns.append(f'{name}_time')
        result_columns.append(f'{name}_params')
        result_columns.append(f'{name}_memMB')
        
    result_df = pd.DataFrame(row_values, columns=result_columns)
    result_df.insert(loc=0, column='label', value=labels)

    return result_df


#fake_dir = './dataset/synthetic/'
#real_dir = './dataset/real/'
myDataset_dir = './dataset/myDataset'
    
#real_result = get_predictions(real_dir)
#fake_result = get_predictions(fake_dir)
myDataset_result = get_predictions(myDataset_dir)

# Compute percentage accuracies for each model and each top-k
n = len(myDataset_result)
summary = {}
for name in model_name:
    top1 = myDataset_result[f'{name}_top1_acc'].sum() / n * 100
    top2 = myDataset_result[f'{name}_top2_acc'].sum() / n * 100
    top3 = myDataset_result[f'{name}_top3_acc'].sum() / n * 100
    summary[name] = {
        'Top-1 Accuracy (%)': round(top1, 2),
        'Top-2 Accuracy (%)': round(top2, 2),
        'Top-3 Accuracy (%)': round(top3, 2)
    }

# Display the summary as a table
summary_df = pd.DataFrame(summary).T
print(summary_df)

              Top-1 Accuracy (%)  Top-2 Accuracy (%)  Top-3 Accuracy (%)
ResNet50                    9.09                0.00                 0.0
VGGNet16                    9.09                0.00                 0.0
InceptionV3                 0.00                0.00                 0.0
ConvNeXt                    9.09                0.00                 0.0
EfficientNet                0.00                4.55                 0.0


In [10]:
#real_result.to_csv('./results/real_result.csv', index = False)
#fake_result.to_csv('./results/fake_result.csv', index = False)
myDataset_result.to_csv('./results/myDataset_result.csv', index = False)